<a href="https://colab.research.google.com/github/sejongmin/AI/blob/main/%EC%8B%A4%EC%8A%B52_2_%ED%83%80%EC%9D%B4%ED%83%80%EB%8B%89_%EC%83%9D%EC%A1%B4%EC%9E%90_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random

## 타이타닉 생존자 예측 문제

[[캐글 링크](https://www.kaggle.com/competitions/titanic/overview)]

In [ ]:
train=pd.read_csv("https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/train.csv")
test=pd.read_csv("https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/test.csv")

In [ ]:
train.head(n=3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [ ]:
print('shape of train data: ', train.shape)
print(train.info())

shape of train data:  (891, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [ ]:
print('shape of test data: ', test.shape)
print(test.info())

shape of test data:  (418, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB
None


## 전처리

* 예측과 관련이 없는 column 삭제
* 범주형 자료 실수로 치환
 - 범주형 자료를 실수 label (1,2,...)로 치환
 - One-hot encoding
* 결측자료 처리
 - 결측치가 있는 데이터 sample을 제거
 - 결측치를 평균, 중위수, 최빈수 등의 방식으로 대체
* 자료의 표준화/정규화


In [ ]:
# PassengerId, Name, Ticket 생존자 예측과 관련이 없다고 생각하고 삭제
train.drop(['PassengerId'], axis=1, inplace=True)
test.drop(['PassengerId'], axis=1, inplace=True)
train.drop(['Name'], axis=1, inplace=True)
test.drop(['Name'], axis=1, inplace=True)
train.drop(['Ticket'], axis=1, inplace=True)
test.drop(['Ticket'], axis=1, inplace=True)

In [ ]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S


In [ ]:
# 범주형 자료 실수로 치환
train['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [ ]:
dic = {'male': 0, 'female': 1}
train['Sex'] = train['Sex'].map(dic)
test['Sex'] = test['Sex'].map(dic)

In [ ]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,0,22.0,1,0,7.2500,NaN,S
1,1,1,1,38.0,1,0,71.2833,C85,C
2,1,3,1,26.0,0,0,7.9250,NaN,S
3,1,1,1,35.0,1,0,53.1000,C123,S
4,0,3,0,35.0,0,0,8.0500,NaN,S


In [ ]:
train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [ ]:
dic = {'S': 0, 'C': 1, 'Q': 2}
train['Embarked'] = train['Embarked'].map(dic)
test['Embarked'] = test['Embarked'].map(dic)

train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,0,22.0,1,0,7.2500,NaN,0.0
1,1,1,1,38.0,1,0,71.2833,C85,1.0
2,1,3,1,26.0,0,0,7.9250,NaN,0.0
3,1,1,1,35.0,1,0,53.1000,C123,0.0
4,0,3,0,35.0,0,0,8.0500,NaN,0.0


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Cabin     204 non-null    object 
 8   Embarked  889 non-null    float64
dtypes: float64(3), int64(5), object(1)
memory usage: 62.8+ KB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    int64  
 2   Age       332 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      417 non-null    float64
 6   Cabin     91 non-null     object 
 7   Embarked  418 non-null    int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 26.2+ KB


In [ ]:
# Age의 결측치 대체
train['Age'].fillna(train['Age'].mean(), inplace=True)
test['Age'].fillna(train['Age'].mean(), inplace=True)

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Cabin     204 non-null    object 
 8   Embarked  889 non-null    float64
dtypes: float64(3), int64(5), object(1)
memory usage: 62.8+ KB


In [ ]:
# Cabin 결측치 너무 많아 삭제
train.drop(['Cabin'], axis=1, inplace=True)
test.drop(['Cabin'], axis=1, inplace=True)

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    float64
dtypes: float64(3), int64(5)
memory usage: 55.8 KB


In [ ]:
# Embarked 결측치 2개 sample 결측치 존재, 두 개 밖에 안되므로 sample drop
train = train.dropna(axis=0)

train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  889 non-null    int64  
 1   Pclass    889 non-null    int64  
 2   Sex       889 non-null    int64  
 3   Age       889 non-null    float64
 4   SibSp     889 non-null    int64  
 5   Parch     889 non-null    int64  
 6   Fare      889 non-null    float64
 7   Embarked  889 non-null    float64
dtypes: float64(3), int64(5)
memory usage: 62.5 KB


In [ ]:
# test 데이터의 Fare 결측치 대체
test['Fare'].fillna(train['Fare'].mean(), inplace=True)

test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    int64  
 2   Age       418 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      418 non-null    float64
 6   Embarked  418 non-null    int64  
dtypes: float64(2), int64(5)
memory usage: 23.0 KB


In [ ]:
# 특징정보, 목푯값 분리
X_train = train.drop('Survived', axis=1)
Y_train = pd.DataFrame(train['Survived'])

X_test = test

In [ ]:
# scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## PyTorch 딥러닝 모델 정의 및 학습

* DataLoader 정의
* 딥러닝 모델 정의
* 손실함수 및 최적화 알고리즘 정의
* 학습 진행 및 예측 수행
* 예측 결과 출력


In [ ]:
import torch
import torch.nn as nn

In [ ]:
X_train_ = torch.from_numpy(X_train).type(torch.float)
Y_train_ = torch.from_numpy(Y_train.values).type(torch.float)

X_test_ = torch.from_numpy(X_test).type(torch.float)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(X_train_, Y_train_)
loader = DataLoader(dataset, batch_size=64)

In [ ]:
X_train.shape

(889, 7)

In [ ]:
linear1 = torch.nn.Linear(X_train.shape[1], 100)
linear2 = torch.nn.Linear(100, 100)
linear3 = torch.nn.Linear(100, 100)
linear4 = torch.nn.Linear(100, 1)

relu = torch.nn.ReLU()

model = nn.Sequential(linear1,relu,linear2,relu,linear3,relu,linear4)

In [ ]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
epochs = 1000
for i in range(epochs):
    for x,y in loader:
        optimizer.zero_grad()
        # Generate Prediction
        preds = model(x)
        # Get the loss and perform backpropagation
        loss = criterion(preds, y)
        loss.backward()
        # Let's update the weights
        optimizer.step()
    print(f"Epoch {i}/{epochs}: Loss: {loss}")

Epoch 0/1000: Loss: 0.5752925276756287
Epoch 1/1000: Loss: 0.39277440309524536
Epoch 2/1000: Loss: 0.3380000591278076
Epoch 3/1000: Loss: 0.32842808961868286
Epoch 4/1000: Loss: 0.3144024610519409
Epoch 5/1000: Loss: 0.30569905042648315
Epoch 6/1000: Loss: 0.3021227717399597
Epoch 7/1000: Loss: 0.2995803654193878
Epoch 8/1000: Loss: 0.29679372906684875
Epoch 9/1000: Loss: 0.29379794001579285
Epoch 10/1000: Loss: 0.2909279465675354
Epoch 11/1000: Loss: 0.2884812653064728
Epoch 12/1000: Loss: 0.28583237528800964
Epoch 13/1000: Loss: 0.28330591320991516
Epoch 14/1000: Loss: 0.280401349067688
Epoch 15/1000: Loss: 0.27782952785491943
Epoch 16/1000: Loss: 0.27426648139953613
Epoch 17/1000: Loss: 0.27188196778297424
Epoch 18/1000: Loss: 0.268443763256073
Epoch 19/1000: Loss: 0.26593342423439026
Epoch 20/1000: Loss: 0.262248694896698
Epoch 21/1000: Loss: 0.2600167393684387
Epoch 22/1000: Loss: 0.2557098865509033
Epoch 23/1000: Loss: 0.25325068831443787
Epoch 24/1000: Loss: 0.24986819922924042


In [ ]:
X_test_.shape

torch.Size([418, 7])

In [ ]:
with torch.no_grad():
    Y_test_pred = model(X_test_)
    Y_test_pred = torch.sigmoid(Y_test_pred)

pd.DataFrame(Y_test_pred.detach().numpy().astype('int')).to_csv('submit.csv', header=False, index=False)